# Load 'bcl2' sentences data

In [2]:
bcl2 = spark.read.csv('raw_data/article-content.txt', sep='\t', inferSchema=True).toDF('id', 'sentences')

In [4]:
bcl2.show(5)

+--------+--------------------+
|      id|           sentences|
+--------+--------------------+
|28386116| we experimentall...|
|28386116| to evaluate the ...|
|28386116|it has been well ...|
|28386116| our bioinformati...|
|28386116| it has been prev...|
+--------+--------------------+
only showing top 5 rows



# Lemmatizing words

In [5]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

In [8]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

def lemmatizer(s):
    words_list = [wordnet_lemmatizer.lemmatize(w) for w in s.split()]
    return(words_list)
lemmatizer_udf = udf(lemmatizer, ArrayType(StringType()))

In [11]:
bcl2_lemm = bcl2.select(bcl2.id, lemmatizer_udf(bcl2.sentences).alias('lemm_words'))
bcl2_lemm.show(5, truncate=False)

+--------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id      |lemm_words                                                                                                                                                                                                                                                                                                                                                                     |
+--------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------